In [1]:
from hydra import initialize, compose
import polars as pl
import pickle
from pathlib import Path
import torch
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from itertools import chain, product
from tqdm import tqdm
import numpy as np
from src.utils import construct_sparse_adj_mat
from src.cross_validation import parse_cd_hit_clusters
import json
import matplotlib.pyplot as plt
from itertools import combinations

In [2]:
with initialize(version_base=None, config_path="../configs/filepaths"):
    cfg = compose(config_name="base")

In [3]:
with open(Path(cfg.data) / "clipzyme" / "enzymemap.json", "r") as f:
    enzymemap = json.load(f)

In [46]:
with open(Path(cfg.data) / "clipzyme" / "cached_enzymemap.p", "rb") as f:
    clip_data = pickle.load(f)

with open("/home/stef/clipzyme/scripts/repro_cache.p", "rb") as f:
    repro_cache = pickle.load(f)

for elt in clip_data:
    rcts, pdts = [side.split(".") for side in elt['reaction_string'].split(">>")]
    elt['reaction_string'] = ".".join(sorted(rcts)) + ">>" + ".".join(sorted(pdts))

for elt in repro_cache:
    elt['reaction_string'] = ".".join(sorted(elt['reactants'])) + ">>" + ".".join(sorted(elt['products']))

In [5]:
type(clip_data)

list

In [23]:
with open("/home/stef/clipzyme/scripts/rule_splits.json", "r") as f:
    rule_splits = json.load(f)

rule_splits = {int(k): v for k, v in rule_splits.items()}

In [7]:
print(Path(cfg.data) / "clipzyme" / "enzymemap.json")


/home/stef/quest_data/hiec/data/clipzyme/enzymemap.json


In [8]:
test_rxn = enzymemap[40]
print("Test reaction:")
for k, v in test_rxn.items():
    print(f"{k}: {v}")

for i, rxn in enumerate(enzymemap):
    if sorted(rxn['reactants']) == sorted(test_rxn['products']) and sorted(rxn['products']) == sorted(test_rxn['reactants']):
        print(f"\nFound reverse reaction and index {i}:")
        for k, v in rxn.items():
            print(f"{k}: {v}")
        # break

Test reaction:
mapped_reactants: ['[CH3:1][CH:2]=[O:3]', '[H+]', '[NH2:4][C:5](=[O:6])[C:7]1=[CH:8][N:9]([C@@H:10]2[O:11][C@H:12]([CH2:13][O:14][P:15](=[O:16])([OH:17])[O:18][P:19](=[O:20])([OH:21])[O:22][CH2:23][C@H:24]3[O:25][C@@H:26]([n:27]4[cH:28][n:29][c:30]5[c:31]([NH2:32])[n:33][cH:34][n:35][c:36]45)[C@H:37]([OH:38])[C@@H:39]3[OH:40])[C@@H:41]([OH:42])[C@H:43]2[OH:44])[CH:45]=[CH:46][CH2:47]1']
mapped_products: ['[CH3:1][CH2:2][OH:3]', '[NH2:4][C:5](=[O:6])[c:7]1[cH:8][n+:9]([C@@H:10]2[O:11][C@H:12]([CH2:13][O:14][P:15](=[O:16])([OH:17])[O:18][P:19](=[O:20])([OH:21])[O:22][CH2:23][C@H:24]3[O:25][C@@H:26]([n:27]4[cH:28][n:29][c:30]5[c:31]([NH2:32])[n:33][cH:34][n:35][c:36]45)[C@H:37]([OH:38])[C@@H:39]3[OH:40])[C@@H:41]([OH:42])[C@H:43]2[OH:44])[cH:45][cH:46][cH:47]1']
reactants: ['CC=O', 'NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](O)[C@@H]3O)[C@@H](O)[C@H]2O)C=CC1', '[H+]']
products: ['CCO', 'NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=O)(

In [9]:
uq_rule_ids = set(elt['rule_id'] for elt in enzymemap)

In [10]:
len(uq_rule_ids)

492

In [11]:
test_rxn = clip_data[0]
print("Test reaction:")
for k, v in test_rxn.items():
    print(f"{k}: {v}")

for i, rxn in enumerate(clip_data):
    test_rcts, test_pdts = [elt.split('.') for elt in test_rxn['reaction_string'].split('>>')]
    text_rev = '.'.join(test_pdts) + '>>' + '.'.join(test_rcts)
    if rxn['reaction_string'] == text_rev:
    # if sorted(rxn['reactants']) == sorted(test_rxn['products']) and sorted(rxn['products']) == sorted(test_rxn['reactants']):
        print(f"\nFound reverse reaction and index {i}:")
        for k, v in rxn.items():
            print(f"{k}: {v}")

Test reaction:
reaction_string: CC=O.NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](O)[C@@H]3O)[C@@H](O)[C@H]2O)C=CC1.[H+]>>CCO.NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](O)[C@@H]3O)[C@@H](O)[C@H]2O)c1
df_row: 2
quality: 0.9917081260364844
reactants: ['[CH3:1][CH:2]=[O:3]', '[NH2:4][C:5](=[O:6])[C:7]1=[CH:8][N:9]([C@@H:10]2[O:11][C@H:12]([CH2:13][O:14][P:15](=[O:16])([OH:17])[O:18][P:19](=[O:20])([OH:21])[O:22][CH2:23][C@H:24]3[O:25][C@@H:26]([n:27]4[cH:28][n:29][c:30]5[c:31]([NH2:32])[n:33][cH:34][n:35][c:36]45)[C@H:37]([OH:38])[C@@H:39]3[OH:40])[C@@H:41]([OH:42])[C@H:43]2[OH:44])[CH:45]=[CH:46][CH2:47]1']
products: ['[CH3:1][CH2:2][OH:3]', '[NH2:4][C:5](=[O:6])[c:7]1[cH:8][n+:9]([C@@H:10]2[O:11][C@H:12]([CH2:13][O:14][P:15](=[O:16])([OH:17])[O:18][P:19](=[O:20])([OH:21])[O:22][CH2:23][C@H:24]3[O:25][C@@H:26]([n:27]4[cH:28][n:29][c:30]5[c:31]([NH2:32])[n:33][cH:34][n:35][c:36]45)[C@H:37]([OH:38])[C@@H:39

In [12]:
rules = [elt['rule_id'] for elt in clip_data]

In [19]:
rxn_eq_entries = set()
rxn_rev_eq_entries = set()
n = len(clip_data)
for i, j in tqdm(combinations(range(len(clip_data)), 2), total=n*(n-1), desc="Checking reaction equivalences"):
    if clip_data[i]['reaction_string'] == clip_data[j]['reaction_string']:
        rxn_eq_entries.add((i, j))
        continue

    rev_i = ">>".join(clip_data[i]['reaction_string'].split('>>')[::-1])
    if rev_i == clip_data[j]['reaction_string']:
        rxn_rev_eq_entries.add((i, j))

Checking reaction equivalences:  50%|█████     | 1074416190/2148832380 [10:22<10:22, 1725234.22it/s] 


In [34]:
eq_leaks = []
rev_eq_leaks = []
for i, j in rxn_eq_entries:
    rule_i = clip_data[i]['rule_id']
    rule_j = clip_data[j]['rule_id']
    split_i = rule_splits[rule_i]
    split_j = rule_splits[rule_j]
    if split_i != split_j:
        eq_leaks.append((i, j, rule_i, rule_j, split_i, split_j))

for i, j in rxn_rev_eq_entries:
    rule_i = clip_data[i]['rule_id']
    rule_j = clip_data[j]['rule_id']
    split_i = rule_splits[rule_i]
    split_j = rule_splits[rule_j]
    if split_i != split_j:
        rev_eq_leaks.append((i, j, rule_i, rule_j, split_i, split_j))

print(f"Number of exact reaction equivalence leaks: {len(eq_leaks)}")
print(f"Number of reverse reaction equivalence leaks: {len(rev_eq_leaks)}")

Number of exact reaction equivalence leaks: 2725
Number of reverse reaction equivalence leaks: 2729


In [35]:
eq_leaks

[(19192, 19197, 388, 417, 'dev', 'test'),
 (41074, 41121, 183, 166, 'test', 'train'),
 (41086, 41131, 183, 166, 'test', 'train'),
 (41059, 41132, 183, 166, 'test', 'train'),
 (13618, 13633, 138, 116, 'test', 'train'),
 (41051, 41108, 183, 166, 'test', 'train'),
 (41092, 41109, 183, 166, 'test', 'train'),
 (41104, 41119, 183, 166, 'test', 'train'),
 (41053, 41130, 183, 166, 'test', 'train'),
 (41026, 41131, 183, 166, 'test', 'train'),
 (40729, 40736, 92, 91, 'test', 'train'),
 (41098, 41117, 183, 166, 'test', 'train'),
 (41071, 41118, 183, 166, 'test', 'train'),
 (41065, 41116, 183, 166, 'test', 'train'),
 (41038, 41117, 183, 166, 'test', 'train'),
 (41050, 41127, 183, 166, 'test', 'train'),
 (39555, 39558, 732, 729, 'test', 'train'),
 (41095, 41114, 183, 166, 'test', 'train'),
 (41044, 41125, 183, 166, 'test', 'train'),
 (39844, 39929, 92, 91, 'test', 'train'),
 (41089, 41112, 183, 166, 'test', 'train'),
 (41009, 41102, 166, 183, 'train', 'test'),
 (41062, 41113, 183, 166, 'test', 'tra

In [37]:
unq_test_pts_leaked = set()
for i, j, rule_i, rule_j, split_i, split_j in eq_leaks + rev_eq_leaks:
    if split_i == 'test' and split_j == 'train':
        unq_test_pts_leaked.add(i)
    if split_j == 'test' and split_i == 'train':
        unq_test_pts_leaked.add(j)

print(f"Number of unique test points leaked: {len(unq_test_pts_leaked)}")

Number of unique test points leaked: 405


In [29]:
for k, v in clip_data[41074].items():
    print(f"{k}: {v}")

reaction_string: O=C(O)O>>O.O=C=O
df_row: 305575
quality: 0.9448373408769448
reactants: ['[O:1]=[C:2]([OH:3])[OH:4]']
products: ['[O:1]=[C:2]=[O:3]', '[OH2:4]']
ec: 4.2.1.1
rowid: 285760
sample_id: A0A0N9FGL8_285760_305575
uniprot_id: A0A0N9FGL8
protein_id: A0A0N9FGL8
bond_changes: [['2', '4', 0.0], ['2', '3', 2.0]]
split: None
protein_refs: ['A0A0N9FGL8']
protein_db: uniprot
rule_id: 183
ec1: 4
ec2: 4.2
ec3: 4.2.1
ec4: 4.2.1.1


In [43]:
for k, v in clip_data[19750].items():
    print(f"{k}: {v}")

reaction_string: CC(C)=CCOP(=O)(O)OP(=O)(O)O.O=C1N[C@H](Cc2c[nH]c3ccccc23)C(=O)N2CCC[C@H]12>>CC(C)=CCc1[nH]c2ccccc2c1C[C@H]1NC(=O)[C@H]2CCCN2C1=O.O=P(O)(O)OP(=O)(O)O
df_row: 154277
quality: 1.0
reactants: ['[CH3:1][C:2]([CH3:3])=[CH:4][CH2:5][O:6][P:7](=[O:8])([OH:9])[O:10][P:11](=[O:12])([OH:13])[OH:14]', '[O:15]=[C:16]1[NH:17][C@H:18]([CH2:19][c:20]2[cH:21][nH:22][c:23]3[cH:24][cH:25][cH:26][cH:27][c:28]23)[C:29](=[O:30])[N:31]2[CH2:32][CH2:33][CH2:34][C@H:35]12']
products: ['[CH3:1][C:2]([CH3:3])=[CH:4][CH2:5][c:21]1[c:20]([CH2:19][C@H:18]2[NH:17][C:16](=[O:15])[C@@H:35]3[N:31]([C:29]2=[O:30])[CH2:32][CH2:33][CH2:34]3)[c:28]2[c:23]([nH:22]1)[cH:24][cH:25][cH:26][cH:27]2', '[OH:6][P:7](=[O:8])([OH:9])[O:10][P:11](=[O:12])([OH:13])[OH:14]']
ec: 2.5.1.106
rowid: 146427
sample_id: Q4WAW7_146427_154277
uniprot_id: Q4WAW7
protein_id: Q4WAW7
bond_changes: [['21', '5', 1.0], ['5', '6', 0.0]]
split: None
protein_refs: ['Q4WAW7']
protein_db: uniprot
rule_id: 294
ec1: 2
ec2: 2.5
ec3: 2.5.1
ec4

In [40]:
set(len(elt['protein_refs']) for elt in clip_data)

{1, 2, 3, 4, 5, 6, 8}

In [42]:
for elt in clip_data:
    if len(elt['protein_refs']) > 1:
        for k, v in elt.items():
            print(f"{k}: {v}")

        break

reaction_string: NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)c1.O=C(O)[C@H](O)[C@@H](O)[C@H](O)[C@H](O)CO>>NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)C=CC1.O=C(O)C(=O)[C@@H](O)[C@H](O)[C@H](O)CO.[H+]
df_row: 19750
quality: 1.0
reactants: ['[NH2:1][C:2](=[O:3])[c:4]1[cH:5][cH:6][cH:7][n+:8]([C@@H:9]2[O:10][C@H:11]([CH2:12][O:13][P:14](=[O:15])([OH:16])[O:17][P:18](=[O:19])([OH:20])[O:21][CH2:22][C@H:23]3[O:24][C@@H:25]([n:26]4[cH:27][n:28][c:29]5[c:30]([NH2:31])[n:32][cH:33][n:34][c:35]45)[C@H:36]([O:37][P:38](=[O:39])([OH:40])[OH:41])[C@@H:42]3[OH:43])[C@@H:44]([OH:45])[C@H:46]2[OH:47])[cH:48]1', '[O:49]=[C:50]([OH:51])[C@H:52]([OH:53])[C@@H:54]([OH:55])[C@H:56]([OH:57])[C@H:58]([OH:59])[CH2:60][OH:61]']
products: ['[NH2:1][C:2](=[O:3])[C:4]1=[CH:48][N:8]([C@@H:9]2[O:10][C@H:11]([CH2:12][O:13][P:14](=[O:15])([OH:16])[O:17][P:18](

Re-check with reproduced data cache

In [47]:
rxn_eq_entries_repro = set()
rxn_rev_eq_entries_repro = set()
n = len(repro_cache)
for i, j in tqdm(combinations(range(len(repro_cache)), 2), total=n*(n-1), desc="Checking reaction equivalences"):
    if repro_cache[i]['reaction_string'] == repro_cache[j]['reaction_string']:
        rxn_eq_entries_repro.add((i, j))
        continue

    rev_i = ">>".join(repro_cache[i]['reaction_string'].split('>>')[::-1])
    if rev_i == repro_cache[j]['reaction_string']:
        rxn_rev_eq_entries_repro.add((i, j))

Checking reaction equivalences:  50%|█████     | 1139386716/2278773432 [11:50<11:50, 1603042.95it/s] 


In [48]:
eq_leaks_repro = []
rev_eq_leaks_repro = []
for i, j in rxn_eq_entries_repro:
    rule_i = repro_cache[i]['rule_id']
    rule_j = repro_cache[j]['rule_id']
    split_i = rule_splits[rule_i]
    split_j = rule_splits[rule_j]
    if split_i != split_j:
        eq_leaks_repro.append((i, j, rule_i, rule_j, split_i, split_j))

for i, j in rxn_rev_eq_entries_repro:
    rule_i = repro_cache[i]['rule_id']
    rule_j = repro_cache[j]['rule_id']
    split_i = rule_splits[rule_i]
    split_j = rule_splits[rule_j]
    if split_i != split_j:
        rev_eq_leaks_repro.append((i, j, rule_i, rule_j, split_i, split_j))

print(f"Number of exact reaction equivalence leaks: {len(eq_leaks_repro)}")
print(f"Number of reverse reaction equivalence leaks: {len(rev_eq_leaks_repro)}")

Number of exact reaction equivalence leaks: 2682
Number of reverse reaction equivalence leaks: 2678


In [49]:
unq_test_pts_leaked_repro = set()
for i, j, rule_i, rule_j, split_i, split_j in eq_leaks_repro + rev_eq_leaks_repro:
    if split_i == 'test' and split_j == 'train':
        unq_test_pts_leaked_repro.add(i)
    if split_j == 'test' and split_i == 'train':
        unq_test_pts_leaked_repro.add(j)

print(f"Number of unique test points leaked: {len(unq_test_pts_leaked_repro)}")

Number of unique test points leaked: 401
